In [1]:
# CONNECT TO DATABASE
import pymongo
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
client = mongo_client["GreekParliamentProceedings"]
index = client["InvertedIndex"]
database = client["Database"]

### Approach 1: Using greek sentiment lexicon (https://github.com/MKLab-ITI/greek-sentiment-lexicon)

In [12]:
import spacy
import pandas as pd
import operator
indexes = {}
df = pd.read_csv('greek_sentiment_lexicon.tsv',sep='\t')
df = df.fillna('N/A')
for index, row in df.iterrows():
    df.at[index, "Term"] = row["Term"].split(' ')[0]
    indexes[df.at[index, "Term"]] = index
    
def main_emotion(speech, id,  indexes):
    results = {}
    text = speech
    subj_scores = {
        'OBJ': 0,
        'SUBJ-': 0.5,
        'SUBJ+': 1,
    }

    emotion_scores = {
        'N/A': 0,
        '1.0': 0.2,
        '2.0': 0.4,
        '3.0': 0.6,
        '4.0': 0.8,
        '5.0': 1,
    }

    polarity_scores = {
        'N/A': 0,
        'BOTH': 0,
        'NEG': -1,
        'POS': 1
    }

    nlp = spacy.load("el_core_news_sm")
    doc = nlp(text)
    matched_tokens = 0
    scorelist = [0 for _ in range(7)]
    for token in doc:
        lemmatized_token = token.lemma_
        if (lemmatized_token in indexes):
            indx = indexes[lemmatized_token]
            pos_flag = False
            for col in ["POS1", "POS2", "POS3", "POS4"]:
                if (token.pos_ == df.at[indx,col]):
                    pos_flag = True
                    break
            if (pos_flag == True):
                match_col_index = [int(s) for s in col if s.isdigit()][0]
                indexlist = ['Subjectivity', 'Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise']
                for i in range(len(scorelist)):
                    if(i==0):
                        if(df.at[indx,indexlist[i]+str(match_col_index)]!='N/A'):
                            scorelist[i] += subj_scores[df.at[indx,indexlist[i]+str(match_col_index)]]
                    else:
                        if(df.at[indx, indexlist[i]+str(match_col_index)]!='N/A' and str(df.at[indx, indexlist[i]+str(match_col_index)])!= '0.0'):
                            scorelist[i] += emotion_scores[str(df.at[indx, indexlist[i]+str(match_col_index)])]
                matched_tokens+=1

    try:
        emotions = {'anger': scorelist[1], 'disgust': scorelist[2], 'fear':scorelist[3], 'happiness':scorelist[4], 'sadness': scorelist[5], 'surprise': scorelist[6]}
        emotion = max(emotions.items(), key=operator.itemgetter(1))[0]
        if (emotions[emotion] == 0):
            results[id] = ['N/A']
        else:
            results[id] = [emotion, str(emotions[emotion]*100/matched_tokens)]
    except:
        print('No matched tokens')
    return results

In [3]:
parties = list(database.distinct("political_party"))
governments_per_party = {}
for party in parties:
    pipeline = [{'$match' : {'political_party':party}},
                    {'$group':{'_id':'$government'}}]
    governments = list(database.aggregate(pipeline))
    governments_per_party[party] = governments

In [14]:
keys = list(governments_per_party.keys())
print(keys.index('λαικη ενοτητα'))

16


In [10]:
print(parties[15])
print(len(parties))

κομμουνιστικο κομμα ελλαδας
27


In [13]:
party_sentiments_per_government = {}
counter = 0
#for each party
for i in range(16, len(keys)):
    print(parties[i])
    party_sentiments_per_government[parties[i]] = {}
    party_sentiments = {}
    #for each government period the party is in the parliament
    for government in governments_per_party[parties[i]]:
        print("     "+government['_id'])
        #get speeches of party for that government
        speeches = database.find({"political_party":parties[i], "government":government['_id']}, {"_id":1, "speech":1})
        for speech in speeches:
            results = main_emotion(speech['speech'], speech['_id'], indexes)
            party_sentiments.update(results)
            
        #count sentiments of all speeches of a party in the specified government
        sent_counter = {'surprise':0, 'happiness':0, 'anger':0, 'disgust':0, 'anger':0, 'fear':0, 'sadness':0}
        for key in party_sentiments.keys():
            if(party_sentiments[key][0]!='N/A'):
                sent_counter[party_sentiments[key][0]] +=1 
                
        party_sentiments_per_government[parties[i]][government['_id']] = sent_counter
        import pickle

        with open('sentiments_per_party_per_government3.pickle', 'wb') as handle:
            pickle.dump(party_sentiments_per_government, handle, protocol=pickle.HIGHEST_PROTOCOL)

λαικη ενοτητα
     ['τσιπρα π. αλεξιου(26/01/2015-27/08/2015)']
2.0
3.0
1.0
1.0
2.0
1.0
1.0
1.0
1.0
3.0
1.0
3.0
1.0
1.0
1.0
3.0
1.0
3.0
2.0
2.0
2.0
3.0
2.0
3.0
4.0
2.0
1.0
1.0
1.0
2.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
4.0
1.0
3.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
2.0
3.0
2.0
3.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
3.0
2.0
4.0
2.0
2.0
2.0
2.0
2.0
3.0
2.0
3.0
1.0
2.0
1.0
1.0
1.0
2.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
4.0
4.0
4.0
1.0
1.0
3.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
1.0
4.0
2.0
2.0
2.0
3.0
2.0
3.0
1.0
2.0
1.0
1.0
5.0
3.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
4.0
1.0
4.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
4.0
4.0
4.0
1.0
1.0
3.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
4.0
1.0
4.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
5.0
1.0
4.0
1.0
1.0
1.0
1.0
1.0
1.0


KeyError: '0.0'